In [1]:
import pandas as pd
import spacy

In [2]:
nlp = spacy.load('en')

/home/katia/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
PLACEHOLDER_NAME = 'PROT_NAME'

## 1. Load data

In [4]:
excerpts = pd.read_excel('data/Sample_Paragraphs.xlsx', 'Sheet1')
excerpts

,Book Title,Paragraph,Character,Author
0,"Rabbit Angstrom: Rabbit, Run",The straight path is made smooth. Mr. Springer...,Mr. Springer,John Updike
1,The Sound and Fury,“What’s the matter?” I says. “I never knew you...,Sarah,William Faulkner
2,Beloved,Sethe moaned. The girl’s fingers were so cool ...,Sethe,Toni Morrison
3,A Son at The Front,"John Campton, the American portrait-painter, s...",John Campton,Edith Wharton
4,The Murder on the Links,It was five minutes past nine when I entered o...,Poirot,Agatha Christie
5,The Left Hand of Darkness,The king seems to be finished with his masonry...,the king,Ursula K. Le Guin
6,The Story of a Great School Master,Of all the men I have met—and I have now had a...,F.W. Sanderson,H. G. Wells
7,Don Quixote,Don Quixote was firmly persuaded that this was...,Don Quixote,Miguel de Cervates
8,Pride and Prejudice,"Not all that Mrs. Bennet, however, with the as...",Mrs. Bennet,Jane Austen


### Get only the text & the protagonist for this chosen book

In [5]:
text = excerpts.loc[8].Paragraph
protagonist = excerpts.loc[8].Character

## 2. Count how many times the name of the character to be re-genders occurs in the texts
- as it is mentioned in the Character column
- as the separate tokens of what is in the Character column, e.g. John Campton - John & Campton & Jon Campton

a. count how many tokens there are in each character column

In [6]:
# we do not split the character name of Mrs. Bennet

# see if spacy removes words which are not named entities, such as `Mrs.` and `the`
doc = nlp(protagonist)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

print('--------')
# unfortunately yes - and in this case we cannot lose `Mrs.` as we need to differentiate from `Mr. Bennet`

Bennet 5 11 PERSON
--------


**Observation**: When `neuralcoref` is imported at the top, the above line's execution kills the current kernel. 
    This might be a memory problem - needs further investigation

#### a. split the text in paragraphs

#### b. count how many times the string, which is the protagonist name, occurs in each paragraph (verify that we have not missed something)

#### c. find the references to the protagonist in each paragraph

In [7]:
# a. split the excerpt into paragraphs when trhe char sequence \n\n is detected
paragraphs = text.split('\n\n')
print('There are', len(paragraphs), 'paragraphs detected.')

There are 4 paragraphs detected.


In [8]:
# b. count how many times the protagonist's name occurs in each paragraph
i = 1
for paragraph in paragraphs:
    name_count = paragraph.count(protagonist)
    print("The protagonist's name occurs", name_count, 'times in paragraph', i)
    print('--------')
    i += 1
    
# replace each occurence of the protagonist's name with a placeholder - to visually see what has been changed
# text = text.replace(protagonist, PLACEHOLDER_NAME)

The protagonist's name occurs 1 times in paragraph 1
--------
The protagonist's name occurs 1 times in paragraph 2
--------
The protagonist's name occurs 0 times in paragraph 3
--------
The protagonist's name occurs 2 times in paragraph 4
--------


**Observation:** yes, this is correct, see the labelled data in this [Google Doc](https://docs.google.com/document/d/1SuEvxH0g0Dkt1kTc4VW_Xkzz1MC784sU22K_Oofac84/edit?ts=5f07b543#)

In [9]:
# c. find the references to the protagonist in each paragraph

# load the co-reference resolution models
import neuralcoref
# nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [ ]:
# process each paragraph one by one
for paragraph in paragraphs:
    doc = nlp(paragraph)